In [ ]:
# Step 1: Importing Libraries
import librosa
import numpy as np
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from IPython.display import Audio
from textblob import TextBlob

In [ ]:
#Step 2: Load Pre-trained Model and Tokenizer
model_name = "facebook/wav2vec2-large-960h"
tokenizer = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 3: Load Audio File
audio_path = '/content/good.unknown'
audio, sample_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz

<ipython-input-15-dd04242599a1>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
# Step 4: Play Audio
Audio(audio, rate=16000)

In [ ]:
# Step 5: Speech to Text
# Tokenization: Convert audio waveform to input tokens
input_values = tokenizer(audio, return_tensors="pt", padding="longest").input_values

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# Model Prediction: Predict logits using the Wav2Vec2 model
with torch.no_grad():
    logits = model(input_values).logits

# Decode Text: Convert predicted IDs to human-readable transcription
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

print("Original Transcription:", transcription)

Original Transcription: HELLO GOOD MORNING HOW ARE YOU


In [ ]:
# Step 6: Auto-correct Spelling
#use a spell-check library like spellchecker
!pip install pyspellchecker
from spellchecker import SpellChecker

In [ ]:
spell = SpellChecker()
corrected_transcription = " ".join([spell.correction(word) for word in transcription.split()])

In [ ]:
# Output the corrected transcription
print("Original Transcription:", transcription)
print("Corrected Transcription:", corrected_transcription)

Original Transcription: HELLO GOOD MORNING HOW ARE YOU
Corrected Transcription: HELLO GOOD MORNING HOW ARE YOU
